# Installing MASE (again)

Run the block below to install MASE in the current Colab runtime

In [2]:
%cd /home/qizhu/Desktop/Work/mase/machop

/home/qizhu/Desktop/Work/mase/machop


# General introduction

In this lab, you will learn how to use the software stack of MASE. There are in total 7 tasks you would need to finish, and 1 optional task.

# Turning you network to a graph

One specific feature of MASE is its capability to transform DL models to a computation graph using the [torch.fx](<https://pytorch.org/docs/stable/fx.html>) framework.


## Use the Transform functionality without CLI

This tutorial describes how to use the MASE transform functionality for a pre-trained model.

## Import related packages and machop

In [3]:
import sys
import logging
import os
sys.path.append('/home/qizhu/Desktop/Work/mase/machop')
from pathlib import Path
from pprint import pprint as pp

In [4]:
from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.passes.graph import (
    save_node_meta_param_interface_pass,
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
)
from chop.tools.get_input import InputGenerator
from chop.tools.checkpoint_load import load_model
from chop.ir import MaseGraph

from chop.models import get_model_info, get_model

set_logging_verbosity("info")

/home/qizhu/miniconda3/envs/mase/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO     Set logging level to info


## Set up the dataset

Here we create a `MaseDataModule` using the `jsc` dataset from lab1. Note the `MaseDataModule` also requires the name of the model you plan to use data module with. In this case it is `jsc-tiny`.

In [5]:
batch_size = 8
model_name = "jsc-tiny"
dataset_name = "jsc"


data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()


## Set up the model 

Here we use the previously trained `jsc-tiny` model in lab 1 as an example.

In [6]:
# 📝️ change this CHECKPOINT_PATH to the one you trained in Lab1
CHECKPOINT_PATH = "/home/qizhu/Desktop/Work/mase/mase_output/jsc-tiny_classification_jsc_2024-02-11/software/training_ckpts/best-v2.ckpt"
model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)

model = load_model(load_name=CHECKPOINT_PATH, load_type="pl", model=model)

INFO     Loaded pytorch lightning checkpoint from /home/qizhu/Desktop/Work/mase/mase_output/jsc-tiny_classification_jsc_2024-02-11/software/training_ckpts/best-v2.ckpt


# Get a dummy data in
With the dataset module and model information, we can grab an input generator.

In [7]:
# get the input generator
input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

# a demonstration of how to feed an input value to the model
dummy_in = next(iter(input_generator))
_ = model(**dummy_in)


## Generate a MaseGraph
We have two forms of passes: transform passes and analysis passes, both of them would require the model to be transferred into a MaseGraph to allow manipulation.

In [11]:
# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)
mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

In [9]:
from chop.passes.graph.analysis import flops_statistics_analysis_pass, bits_statistics_analysis_pass

In [13]:
flops_statistics_analysis_pass(mg)
bits_statistics_analysis_pass(mg)

1320
655360


(<chop.ir.graph.mase_graph.MaseGraph at 0x7f5f006e68c0>,
 {'total_flops': 655360.0})

In [21]:
for node in mg.fx_graph.nodes:
    # mase_meta = node.meta["mase"].parameters
    # mase_op = mase_meta["common"]["mase_op"]
    # print(mase_op)
    print(node)

x
seq_blocks_0
seq_blocks_1
seq_blocks_2
seq_blocks_3
output


## Running an Analysis pass
Analysis pass DOES NOT change the graph

The following analysis passes are essential to prepare the graph for other passes

In [10]:
mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

We will first run a simple graph analysis to understand the structure of the model.

In [18]:
# report graph is an analysis pass that shows you the detailed information in the graph
from chop.passes.graph import report_graph_analysis_pass
_ = report_graph_analysis_pass(mg)

graph():
    %x : [num_users=2] = placeholder[target=x]
    %seq_blocks_0 : [num_users=1] = call_module[target=seq_blocks.0](args = (%x,), kwargs = {})
    %seq_blocks_1 : [num_users=1] = call_module[target=seq_blocks.1](args = (%seq_blocks_0,), kwargs = {})
    %seq_blocks_2 : [num_users=1] = call_module[target=seq_blocks.2](args = (%seq_blocks_1,), kwargs = {})
    %seq_blocks_3 : [num_users=1] = call_module[target=seq_blocks.3](args = (%seq_blocks_2,), kwargs = {})
    %fc1 : [num_users=1] = call_module[target=fc1](args = (%x,), kwargs = {})
    %add : [num_users=1] = call_function[target=operator.add](args = (%seq_blocks_3, %fc1), kwargs = {})
    return add
Network overview:
{'placeholder': 1, 'get_attr': 0, 'call_function': 1, 'call_method': 0, 'call_module': 5, 'output': 1}
Layer types:
[BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(inplace=True), Linear(in_features=16, out_features=5, bias=True), ReLU(inplace=True), Linear(in_features=16,

## Running another Analysis pass: Profile statistics

The pass `profile_statistics_analysis_pass` collects statistics of parameters and activations, and save them to node's metadata.

Here is a list of all the supported statistics. Refer to the `__init__` of statistic classes in `chop.passes.analysis.statistical_profiler.stat` to check the args each stat class takes.

This is a more complex analysis than the previous pass, and thus it would require you to pass in additional arguments for this pass.

### Example: the range of weights & input activations of nodes

Say we want to collect the tensor-wise min-max range of the 1st `torch.nn.Linear` nodes' weights & bias, and the channel-wise 97% quantile min-max of the 1st `torch.nn.Linear` nodes' input activations. We can do the following:

In [21]:

pass_args = {
    "by": "type",                                                            # collect statistics by node name
    "target_weight_nodes": ["linear"],                                       # collect weight statistics for linear layers
    "target_activation_nodes": ["relu"],                                     # collect activation statistics for relu layers
    "weight_statistics": {
        "variance_precise": {"device": "cpu", "dims": "all"},                # collect precise variance of the weight
    },
    "activation_statistics": {
        "range_quantile": {"device": "cpu", "dims": "all", "quantile": 0.97} # collect 97% quantile of the activation range
    },
    "input_generator": input_generator,                                      # the input generator for feeding data to the model
    "num_samples": 32,                                                       # feed 32 samples to the model
}

We can use the `report_node_meta_param_analysis_pass` to inspect the collected statistics.

In [22]:
mg, _ = profile_statistics_analysis_pass(mg, pass_args)
mg, _ = report_node_meta_param_analysis_pass(mg, {"which": ("software",)})

Profiling act statistics: 100%|██████████| 4/4 [00:00<00:00, 277.56it/s]
INFO     Inspecting graph [add_common_meta_param_analysis_pass]
INFO     
+--------------+---------------+---------------------+--------------+-----------------------------------------------------------------------------------------+
| Node name    | Fx Node op    | Mase type           | Mase op      | Software Param                                                                          |
+==============+===============+=====================+==============+=========================================================================================+
| x            | placeholder   | placeholder         | placeholder  | {'results': {'data_out_0': {'stat': {}}}}                                               |
+--------------+---------------+---------------------+--------------+-----------------------------------------------------------------------------------------+
| seq_blocks_0 | call_module   | module              

## Running a Transform pass: Quantisation

As its name suggests, the transform pass would modify the `MaseGraph`.
Similar to the previous analysis pass example, we would need to first declare the configuration for the pass.

In [22]:
pass_args = {
    "by": "type",
    "default": {"config": {"name": None}},
    "linear": {
        "config": {
            "name": "integer",
            # data
            "data_in_width": 8,
            "data_in_frac_width": 4,
            # weight
            "weight_width": 8,
            "weight_frac_width": 4,
            # bias
            "bias_width": 8,
            "bias_frac_width": 4,
        }
    },
    "conv1d": {
        "config": {
            "name": "integer",
            # data
            "data_in_width": 8,
            "data_in_frac_width": 4,
            # weight
            "weight_width": 8,
            "weight_frac_width": 4,
            # bias
            "bias_width": 8,
            "bias_frac_width": 4,
        }
    
    },
}

We can then proceed to apply the transformation, in this case, we kept the original graph on purpose, so that we can print a `diff`.

In [23]:
from chop.passes.graph.transforms import (
    quantize_transform_pass,
    summarize_quantization_analysis_pass,
)
from chop.ir.graph.mase_graph import MaseGraph


ori_mg = MaseGraph(model=model)
ori_mg, _ = init_metadata_analysis_pass(ori_mg, None)
ori_mg, _ = add_common_metadata_analysis_pass(ori_mg, {"dummy_in": dummy_in})

mg, _ = quantize_transform_pass(mg, pass_args)
summarize_quantization_analysis_pass(ori_mg, mg, save_dir="quantize_summary")

INFO     Quantized graph histogram:
INFO     
| Original type   | OP           |   Total |   Changed |   Unchanged |
|-----------------+--------------+---------+-----------+-------------|
| BatchNorm1d     | batch_norm1d |       1 |         0 |           1 |
| Linear          | linear       |       1 |         1 |           0 |
| ReLU            | relu         |       2 |         0 |           2 |
| output          | output       |       1 |         0 |           1 |
| x               | placeholder  |       1 |         0 |           1 |


In [24]:
inputs, labels = next(iter(data_module.val_dataloader()))
mg.model(inputs)

tensor([[1.5859, 0.0000, 0.0000, 0.0000, 5.7695],
        [0.9414, 0.0000, 3.5234, 3.3438, 1.5547],
        [0.8086, 2.2812, 1.6211, 1.7109, 0.0000],
        [1.3633, 0.0000, 0.0000, 1.9570, 1.2695],
        [4.2539, 1.0703, 0.0000, 0.0000, 0.0000],
        [1.6328, 3.8281, 2.8867, 2.3125, 0.0000],
        [0.0000, 0.0000, 0.1094, 3.1289, 2.4336],
        [0.0000, 0.8828, 3.7891, 4.1055, 1.4922]], grad_fn=<ReluBackward0>)

In [18]:
old_mg = MaseGraph(model)

In [19]:
inputs, labels = next(iter(data_module.val_dataloader()))
old_mg.model(inputs)

tensor([[  3.7678,   1.7272, -16.1090, -17.6071,   7.3943],
        [ -1.5366,  -1.3909,   1.0792,   0.7184,  -0.9956],
        [ -0.9890,   0.9364,   0.2796,   0.0480,  -3.5545],
        [ -1.0388,  -0.9393,  -1.7128,   1.2209,  -1.6869],
        [  3.8044,  -0.4426,  -3.2139,  -1.9470,  -1.3612],
        [ -1.0286,   1.6458,  -0.9528,   0.5494,  -4.1782],
        [ -2.3108,  -1.6260,  -4.4452,  -0.1040,   1.9220],
        [ -4.5382,  -0.6942,   1.6112,   1.5397,  -1.3274]],
       grad_fn=<AddmmBackward0>)

In [20]:
flops_statistics_analysis_pass(mg)
bits_statistics_analysis_pass(mg)

1320


KeyError: 'weight'

In [17]:
for (i, node) in enumerate(mg.fx_graph.nodes):
    mase_meta = node.meta["mase"].parameters
    mase_op = mase_meta["common"]["mase_op"]
    mase_type = mase_meta["common"]["mase_type"]
    print(mase_meta['common']['args'].keys())

dict_keys([])
dict_keys(['data_in_0', 'weight', 'bias'])
dict_keys(['data_in_0'])
dict_keys(['data_in_0', 'weight', 'bias'])
dict_keys(['data_in_0'])
dict_keys([])




# Exercises:

We have now seen how to:
1. Set up a dataset
2. Set up a model
3. Generate a `MaseGraph` from the model
4. Run Analysis and Transform passes on the `MaseGraph`

Now consider the following problems:

1. Explain the functionality of `report_graph_analysis_pass` and its printed jargons such as `placeholder`, `get_attr` ... You might find the doc of [torch.fx](https://pytorch.org/docs/stable/fx.html) useful.

2. What are the functionalities of `profile_statistics_analysis_pass` and `report_node_meta_param_analysis_pass` respectively?

## MASE OPs and MASE Types

MASE is designed to be a very high-level intermediate representation (IR), this is very different from the classic [LLVM IR](https://llvm.org/docs/LangRef.html) that you might be familiar with.

The following MASE Types are available:
(Note from Aaron: do we have a page somewhere that have summarized this?)


## A deeper dive into the quantisation transform

3. Explain why only 1 OP is changed after the `quantize_transform_pass` .

4. Write some code to traverse both `mg` and `ori_mg`, check and comment on the nodes in these two graphs. You might find the source code for the implementation of `summarize_quantization_analysis_pass` useful.

5. Perform the same quantisation flow to the bigger JSC network that you have trained in lab1. You must be aware that now the `pass_args` for your custom network might be different if you have used more than the `Linear` layer in your network.

6. Write code to show and verify that the weights of these layers are indeed quantised. You might need to go through the source code of the implementation of the quantisation pass and also the implementation of the [Quantized Layers](../../machop/chop/passes/transforms/quantize/quantized_modules/linear.py) .

## The command line interface

The same flow can also be executed on the command line throw the `transform` action.

```bash
# make sure you have the same printout
pwd
# it should show
# your_dir/mase-tools/machop

# enter the following command
./ch transform --config configs/examples/jsc_toy_by_type.toml --task cls --cpu=0
```
7. Load your own pre-trained JSC network, and perform perform the quantisation using the command line interface.

## \[Optional] Write your own pass

Many examples of existing passes are in the [source code](../..//machop/chop/passes/__init__.py), the [test files](../../machop/test/passes) for these passes also contain useful information on helping you to understand how these passes are used.

Implement a pass to count the number of FLOPs (floating-point operations) and BitOPs (bit-wise operations).